##### 4.1

In [1]:
from statsmodels.stats.power import TTestIndPower
from math import ceil
power_analysis = TTestIndPower()
sample_size = ceil(power_analysis.solve_power(effect_size=0.5, power=0.8, alpha=0.05))
print(f'Sample Size: {sample_size}')

Sample Size: 64


##### 4.2

In [2]:
import numpy as np
from scipy.stats import ttest_ind

rng = np.random.default_rng(42)
sig = 0.05
reject_count = 0
experiment_count = 0

def generate_other_sample(sample):
    new_sample = rng.uniform(size=sample_size)
    new_sample = new_sample - new_sample.mean()  # Center the sample around its mean
    new_sample = sample.mean() + (sample.std() / new_sample.std()) * new_sample  # Scale to have the same std as sample1
    new_sample = new_sample + (sample.mean() - new_sample.mean())  # Adjust the mean to match sample1
    return new_sample

def generate_samples(sample_size):
    sample1 = rng.uniform(0, 1, sample_size)
    sample2 = generate_other_sample(sample1)
    print(f'mean1: {sample1.mean()}, mean2: {sample2.mean()}')
    print(f'std1: {sample1.std()}, std2: {sample2.std()}')
    return sample1, sample2

def perform_ttest(sample_size, sig):
    s1, s2 = generate_samples(sample_size)
    _, p = ttest_ind(s1, s2)
    return p < sig

def perform_experiment(sample_size, sig, test):
    global experiment_count
    experiment_count += 1
    if test(sample_size, sig):
        global reject_count
        reject_count += 1

for _ in range(1000):
    perform_experiment(sample_size, sig, perform_ttest)
print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')
print(f'Type 1 error rate: {reject_count / experiment_count}')

mean1: 0.5259108585387304, mean2: 0.5259108585387304
std1: 0.27472646385863203, std2: 0.274726463858632
mean1: 0.5143295780023751, mean2: 0.5143295780023751
std1: 0.2960450459080213, std2: 0.2960450459080213
mean1: 0.4373085530062869, mean2: 0.4373085530062869
std1: 0.29208044560366886, std2: 0.29208044560366886
mean1: 0.5223625662755473, mean2: 0.5223625662755473
std1: 0.32569067350248104, std2: 0.32569067350248104
mean1: 0.479674856131185, mean2: 0.479674856131185
std1: 0.28837339160948267, std2: 0.28837339160948267
mean1: 0.4975067147619217, mean2: 0.49750671476192176
std1: 0.2840002633540976, std2: 0.28400026335409767
mean1: 0.4853714789306094, mean2: 0.4853714789306094
std1: 0.301894607394829, std2: 0.301894607394829
mean1: 0.4994264638273686, mean2: 0.4994264638273686
std1: 0.2923459948475228, std2: 0.29234599484752277
mean1: 0.5468187656196707, mean2: 0.5468187656196707
std1: 0.2671818020581405, std2: 0.2671818020581405
mean1: 0.46130505625130447, mean2: 0.46130505625130447
std1

mean1: 0.5341932177470767, mean2: 0.5341932177470767
std1: 0.2580976440908646, std2: 0.25809764409086466
mean1: 0.4715723482059845, mean2: 0.47157234820598454
std1: 0.27974943353490095, std2: 0.27974943353490095
mean1: 0.48654036137480905, mean2: 0.48654036137480905
std1: 0.3090562867329, std2: 0.3090562867329
mean1: 0.48458029977654377, mean2: 0.4845802997765437
std1: 0.2654243106377751, std2: 0.2654243106377751
mean1: 0.4850215236582469, mean2: 0.4850215236582469
std1: 0.2865964663072346, std2: 0.2865964663072346
mean1: 0.48634223296716594, mean2: 0.48634223296716594
std1: 0.27138978315364626, std2: 0.27138978315364626
mean1: 0.44545144209803245, mean2: 0.44545144209803245
std1: 0.3052606232500003, std2: 0.3052606232500003
mean1: 0.4579368151411637, mean2: 0.4579368151411637
std1: 0.26540096311394384, std2: 0.26540096311394384
mean1: 0.5352466662281349, mean2: 0.5352466662281349
std1: 0.28839547708946667, std2: 0.28839547708946667
mean1: 0.5208171425169118, mean2: 0.5208171425169118


In [3]:
effect_size = 0.5
experiment_count = 0
reject_count = 0

def perform_ttest_with_effect_size(sample_size, sig):
    s1 = rng.uniform(0,1,sample_size)
    s2 = rng.uniform(0,1,sample_size)
    s2 += effect_size
    _, p = ttest_ind(s1, s2)
    return p < sig

for _ in range(1000):
    perform_experiment(sample_size, sig, perform_ttest_with_effect_size)
print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')

Experiment count: 1000
Reject count: 1000


In [4]:
experiment_count = 0
reject_count = 0

def generate_cauchy_samples(sample_size):
    cauchy1 = rng.standard_cauchy(size=sample_size)
    cauchy2 = rng.standard_cauchy(size=sample_size)
    cauchy2 = cauchy2 - cauchy2.mean()  # Center the sample around its mean
    cauchy2 = cauchy1.mean() + (cauchy1.std() / cauchy2.std()) * cauchy2  # Scale to have the same std as sample1
    cauchy2 = cauchy2 + (cauchy1.mean() - cauchy2.mean())  # Adjust the mean to match sample1
    return cauchy1, cauchy2

def cauchy_ttest(sample_size, sig):
    c1, c2 = generate_cauchy_samples(sample_size)
    _, p = ttest_ind(c1, c2)
    return p < sig

for _ in range(1000):
    perform_experiment(sample_size, sig, cauchy_ttest)

print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')

Experiment count: 1000
Reject count: 0


In [5]:
experiment_count = 0
reject_count = 0

def cauchy_ttest_effect_size(sample_size, sig):
    c1, c2 = rng.standard_cauchy(size=sample_size), rng.standard_cauchy(size=sample_size)
    c2 += effect_size
    _, p = ttest_ind(c1, c2)
    return p < sig

for _ in range(1000):
    perform_experiment(sample_size, sig, cauchy_ttest_effect_size)

print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')

Experiment count: 1000
Reject count: 37


In [6]:
from scipy.stats import mannwhitneyu

experiment_count = 0
reject_count = 0

def cauchy_mannwhitnyu(sample_size, sig):
    c1, c2 = rng.standard_cauchy(size=sample_size), rng.standard_cauchy(size=sample_size)
    c2 += effect_size
    _, p = mannwhitneyu(c1, c2)
    return p < sig

for _ in range(1000):
    perform_experiment(sample_size, sig, cauchy_mannwhitnyu)

print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')

Experiment count: 1000
Reject count: 328
